<a href="https://colab.research.google.com/github/JSJeong-me/GPT-Agent/blob/main/Multimodal/omni_chatting_for_math.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Omni Chatting for Math with Qwen2.5-Omni

This notebook demonstrates how to use Qwen2.5-Omni to chat about math content in a audio and video stream.

In [ ]:
!pip install git+https://github.com/huggingface/transformers@v4.51.3-Qwen2.5-Omni-preview
!pip install qwen-omni-utils
!pip install openai
!pip install flash-attn --no-build-isolation

In [ ]:
from qwen_omni_utils import process_mm_info

# @title inference function
def inference(video_path):
    messages = [
        {"role": "system", "content": [{"type": "text", "text": "You are Qwen, a virtual human developed by the Qwen Team, Alibaba Group, capable of perceiving auditory and visual inputs, as well as generating text and speech."}]},
        {"role": "user", "content": [
                {"type": "video", "video": video_path},
            ]
        },
    ]
    text = processor.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    # image_inputs, video_inputs = process_vision_info([messages])
    audios, images, videos = process_mm_info(messages, use_audio_in_video=True)
    inputs = processor(text=text, audio=audios, images=images, videos=videos, return_tensors="pt", padding=True, use_audio_in_video=True)
    inputs = inputs.to(model.device).to(model.dtype)

    output = model.generate(**inputs, use_audio_in_video=True, return_audio=True)

    text = processor.batch_decode(output[0], skip_special_tokens=True, clean_up_tokenization_spaces=False)
    audio = output[1]
    return text, audio

Load model and processors.

In [ ]:
import torch
from transformers import Qwen2_5OmniForConditionalGeneration, Qwen2_5OmniProcessor

model_path = "Qwen/Qwen2.5-Omni-7B"
model = Qwen2_5OmniForConditionalGeneration.from_pretrained(
    model_path,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    attn_implementation="flash_attention_2",
)
processor = Qwen2_5OmniProcessor.from_pretrained(model_path)

In [ ]:
import librosa
import audioread

from IPython.display import Video
from IPython.display import Audio

#### Omni Chatting

In [ ]:
video_path = "https://qianwen-res.oss-cn-beijing.aliyuncs.com/Qwen2.5-Omni/math.mp4"

display(Video(video_path, width=640, height=360))
display(Audio(librosa.load(audioread.ffdec.FFmpegAudioFile(video_path), sr=16000)[0], rate=16000))

## Use a local HuggingFace model to inference.
response, audio  = inference(video_path)
print(response[0])

display(Audio(audio, rate=24000))

/usr/local/lib/python3.11/dist-packages/librosa/core/audio.py:172: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


/usr/local/lib/python3.11/dist-packages/librosa/core/audio.py:172: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)
qwen-vl-utils using torchvision to read video.
